# Installation
I'm using ``mayavi``, but its ``pip install`` is currently broken.
I don't want to use python 2.7, so here is the workaround through github.
```console
pip install vtk traits traitsui pyface envisage apptools pyqt5
git clone https://github.com/enthought/mayavi.git
cd mayavi
python setup.py install
```
Now add ``jupyter-notebook`` support
```console
jupyter nbextension install --py mayavi --user
jupyter nbextension enable --py mayavi --user

```
Finally install ``xarray``
```console
conda install xarray dask netCDF4 bottleneck
```
@Bonnie: I'm having trouble installing it on SciServer. [Here is the installation webpage.](https://docs.enthought.com/mayavi/mayavi/installation.html)

# To do:
* Select domain and think about adding eulerian fields (SSH, DSO interface, ...)
* Create side-by-side figures
* Implement movie generation (fly over)

# Import useful modules

In [1]:
import xarray as xr
import numpy as np
from mayavi import mlab
from tvtk.pyface.light_manager import CameraLight
import scipy.io as sio

# Define domain

In [2]:
lon0 = -35.2164
lon1 = -22.0220
lat0 =  64.4609
lat1 =  66.8481

# Bathymetry

In [3]:
Depth = xr.open_dataset('../../data/oceanography/Depth.nc')
Depth = Depth.sel(X = slice(lon0,lon1),
                  Y = slice(lat0,lat1)).to_array()
x_eul = Depth['X'].values
y_eul = Depth['Y'].values
bathy = np.transpose(-Depth.values.squeeze()/1.E3) # km

# Particles

In [4]:
locals().update(sio.loadmat('../../data/oceanography/particles.mat'))
x_lag = f_lons_end
y_lag = f_lats_end
z_lag = f_deps_end/1.E3 # km
T_lag = f_TT_end
S_lag = f_SS_end
# Mask domain
mask_x = np.logical_and(x_lag>lon0,x_lag<lon1)
mask_y = np.logical_and(y_lag>lat0,y_lag<lat1)
mask   = np.logical_and(mask_x,mask_y)
# Apply mask
x_lag = x_lag[mask]
y_lag = y_lag[mask]
z_lag = z_lag[mask]
T_lag = T_lag[mask]
S_lag = S_lag[mask]

# Plot: use the window that pops out to choose camera settings

In [5]:
fig = mlab.figure(bgcolor=(0,0,0),size=(1920/2, 1080))
mlab.clf()

# ----------
# Bathymetry
# ----------
surf = mlab.surf(x_eul, y_eul, bathy, colormap='copper',
                                      vmax = np.floor(np.amax(bathy)),
                                      vmin = np.ceil(np.amin(bathy)),
                                      warp_scale="auto")
# warp_scale="auto" automatically rescale and moves the camera. 
# There isn't the same option for points3d.
# store camera poistion, then reposition
scale    = surf.actor.actor.scale
position = surf.actor.actor.position
view     = mlab.view()
roll     = mlab.roll()

# ---------
# Particles
# ---------
points3d = mlab.points3d(x_lag, y_lag, z_lag, T_lag, colormap="bwr", scale_factor=0.08)
points3d.glyph.scale_mode = 'scale_by_vector' # Only change color
mlab.colorbar(object=points3d,title='Temperature °C',nb_labels=3)
# Rescale and reposition
points3d.actor.actor.scale = scale
points3d.actor.actor.position = position
mlab.view(*view)
mlab.roll(roll)

# Show image
mlab.show()